In [15]:
import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline
from matplotlib.dates import DateFormatter, WeekdayLocator, DayLocator, MONDAY

import mysql.connector
from mysql.connector import errorcode
from ittutils import ittconnection, get_resampled, get_raw_price, get_raw_volume, classification_dataset_from_ts

# Create sql for modifieng signals

In [16]:


# change in the real db the sign of trend
update_1_query = (
    " UPDATE signal_signal SET trend = -1 WHERE `signal` = 'RSI' and `rsi_value` > 50 "
)

update_2_query = (
    " UPDATE signal_signal SET trend = 1 WHERE `signal` = 'RSI' and `rsi_value` < 50 "
)

update_3_query = (
    " UPDATE signal_signal SET trend = -1 WHERE `signal` = 'RSI_Cumulative' and `rsi_value` > 50 "
)

update_4_query = (
    " UPDATE signal_signal SET trend = 1 WHERE `signal` = 'RSI_Cumulative' and `rsi_value` < 50 "
)




# Update Staging DB

In [31]:
db_connection = ittconnection('prodcopy')
cursor = db_connection.cursor()
#cursor.execute(update_1_query)
#cursor.execute(update_2_query)
#cursor.execute(update_3_query)
#cursor.execute(update_4_query)
db_connection.commit()
cursor.close()
db_connection.close()

In [32]:
db_connection = ittconnection('prodcopy')
cursor = db_connection.cursor()

#query = (" SELECT * FROM signal_signal WHERE `signal` = 'RSI' or `signal` = 'RSI_Cumulative' ")
#query = (" SELECT * FROM signal_signal WHERE `signal` = 'RSI_Cumulative' and `rsi_value` > 50 and `trend` = 1 ")
query = (" SELECT * FROM signal_signal WHERE id=24743")

signals_df = pd.read_sql(query, con=db_connection)

# make index a timestamp relative_strength
#signals_df['timestamp'] = pd.to_datetime(signals_df['timestamp'], unit='s')
#signals_df.index = pd.DatetimeIndex(signals_df.timestamp)

#signals_df.sort_index(inplace=True)

signals_df[['timestamp','id','source','transaction_currency','signal','horizon','resample_period','trend', 'rsi_value']]



,timestamp,id,source,transaction_currency,signal,horizon,resample_period,trend,rsi_value
0,1.511780e+09,24743,0,LTC,RSI,0,60,-1,72.957867


# Update Prod DB

In [34]:
db_connection_prod = ittconnection('prod')
cursor_prod = db_connection_prod.cursor()
cursor_prod.execute(update_1_query)
cursor_prod.execute(update_2_query)
cursor_prod.execute(update_3_query)
cursor_prod.execute(update_4_query)
db_connection_prod.commit()
cursor_prod.close()
db_connection_prod.close()

# check if update went well

In [35]:
db_connection_prod = ittconnection('prod')
cursor_prod = db_connection_prod.cursor()
query = (" SELECT * FROM signal_signal WHERE `signal` = 'RSI' or `signal` = 'RSI_Cumulative' ")

signals_df = pd.read_sql(query, con=db_connection_prod)

# make index a timestamp relative_strength
signals_df['timestamp'] = pd.to_datetime(signals_df['timestamp'], unit='s')
signals_df.index = pd.DatetimeIndex(signals_df.timestamp)

signals_df.sort_index(inplace=True)

signals_df[['timestamp','id','source','signal','horizon','resample_period','trend', 'rsi_value']]



,timestamp,id,source,signal,horizon,resample_period,trend,rsi_value
timestamp,,,,,,,,
2017-11-27 10:51:34.197150,2017-11-27 10:51:34.197150,24738,0,RSI,0,60,1,20.533055
2017-11-27 10:52:05.861970,2017-11-27 10:52:05.861970,24739,0,RSI,0,60,1,26.149210
2017-11-27 10:52:05.861970,2017-11-27 10:52:05.861970,24740,0,RSI,0,60,1,15.404603
2017-11-27 10:53:38.093490,2017-11-27 10:53:38.093490,24741,0,RSI,0,60,1,19.703279
2017-11-27 10:53:38.093490,2017-11-27 10:53:38.093490,24743,0,RSI,0,60,-1,72.957867
2017-11-27 10:53:38.093490,2017-11-27 10:53:38.093490,24744,0,RSI,0,60,1,20.548210
2017-11-27 10:55:42.010600,2017-11-27 10:55:42.010600,24746,0,RSI,0,60,1,18.211466
2017-11-27 10:55:42.010600,2017-11-27 10:55:42.010600,24748,0,RSI,0,60,-1,76.534889
2017-11-27 10:55:42.010600,2017-11-27 10:55:42.010600,24750,0,RSI,0,60,1,19.773801
